In [1]:
import JupyterNotebooksLib as slicernb
import slicer
import os 
from DICOMLib import DICOMUtils
import numpy as np
import zipfile

from glob import glob
from natsort import natsorted
from tqdm import tqdm

# Set image viewer size to 50% screen size
# slicernb.AppWindow.setWindowSize(scale=0.5)
# slicernb.AppWindow.setContents("viewers")

# Clear scene
slicer.mrmlScene.Clear(False)

# Unzip and organize files 

In [2]:
#input folder 
input_folder = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\Segmentation database"
intermediary_folder = "./temp_unzip"

if  not os.path.exists(intermediary_folder):
    os.mkdir(intermediary_folder)

In [3]:
# for zip_file in os.listdir(input_folder):
#     if "(" in zip_file or ")" in zip_file:
#         continue;
#     zip_path = f"{input_folder}/{zip_file}"
#     Case_id = zip_file.split("_")[0]

#     if "_segmented_structures" in zip_file: 
#         out_path = f"{intermediary_folder}/{Case_id}_segmentations"
#     else:
#         out_path = f"{intermediary_folder}/{Case_id}_volumes"
        
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                     zip_ref.extractall(out_path)

# organize file names throw out unuasable cases 

In [4]:
input_cases = [] 


volumes_paths = list(natsorted(glob(os.path.join(intermediary_folder,  "*"+ f"_volumes"))))

for vol in volumes_paths: 

    seg_path = vol.replace ("_volumes","_segmentations")

    if (os.path.exists(seg_path)):
        input_cases.append ({"vol":vol,"seg":seg_path})
    else:
        print ("cannot find matching segmentation case will be skipped")



cannot find matching segmentation case will be skipped
cannot find matching segmentation case will be skipped


In [5]:
input_cases[0]["vol"].split("\\")[-1].split("_")[0]

'304846'

# using slicer to convert to nii

### will need to alter segmentation nodes to reflect changes in classes you are trying to segment

In [6]:
nii_path = "./nii_ds_2"
nii_path_volumes = f"{nii_path}/Volumes"
nii_path_segmentations = f"{nii_path}/Masks"

try:
    os.mkdir(nii_path)
    os.mkdir(nii_path_volumes)
    os.mkdir(nii_path_segmentations)
except:
    print ("these directories already exist")

these directories already exist


In [7]:
def get_model_class_from_file (file): 
    if "tooth" in file: 
        #print (file.split("tooth_")[1].split("_")[0])

        if (".stl" in file):
            return "tooth_" + file.split("tooth_")[1].split("_")[0]
        else:
            return "tooth_" + file.split("tooth-")[1].split("-")[0].split("_")[0]

    else: 
        # CaseId = file.split("_") [-1].split(".)
        # #print (file.split(f"{CaseId}_")[-1].split("_")[0])
        if (".stl" in file):
            return file.split("_") [-1].split(".")[0]
        else:
            return file.split("-") [-1].split("_")[0]

In [8]:
get_model_class_from_file ("576547P524654_576547_2018874805-cbct-tooth-13_decimated")
#"304846_canal.stl".split("_")

'tooth_13'

In [9]:
"tooth_11".strip("tooth_")

'11'

In [10]:
lable_map = {
    "background":0,
    "tooth_11":1,
    "tooth_12":2,
    "tooth_13":3,
    "tooth_14":4,
    "tooth_15":5,
    "tooth_16":6,
    "tooth_17":7,
    "tooth_18":8,
    # "tooth_21":9,
    # "tooth_22":10,
    # "tooth_23":11,
    # "tooth_24":12,
    # "tooth_25":13,
    # "tooth_26":14,
    # "tooth_27":15,
    # "tooth_28":16,

}

In [11]:
def convert_to_nnunet_lable_tooth (segmentationNodes,referenceVolumeNode):

    all_segment_np = [] 
    for lable, node in segmentationNodes.items():
        name = node.GetName()
    
        segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
        segmentationLblmap.SetName(name+"_lablemap")
        slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(node, segmentationLblmap, referenceVolumeNode)
        mask = slicer.util.arrayFromVolume(segmentationLblmap)
        mask[mask > 0] = lable_map[lable]
    
        all_segment_np.append(mask.copy())



    final_out = all_segment_np[0] 
    for i in range (1,len(all_segment_np)):
        indices  = np.where (final_out <1)
        final_out [indices] = all_segment_np[i][indices]

    segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(next(iter(segmentationNodes.values())), segmentationLblmap, referenceVolumeNode)
    
    slicer.util.updateVolumeFromArray(segmentationLblmap, final_out)

    return segmentationLblmap

    

In [12]:
def convert_to_nnunet_lable_macro_structure (macroNode,referenceVolumeNode):


    name = macroNode.GetName()
    segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    segmentationLblmap.SetName(name+"_lablemap")
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(macroNode, segmentationLblmap, referenceVolumeNode)
    mask = slicer.util.arrayFromVolume(segmentationLblmap)
    mask[mask > 0] = 1

    # labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    #slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(macroNode, segmentationLblmap, referenceVolumeNode)
    slicer.util.updateVolumeFromArray(segmentationLblmap, mask)
    

    return segmentationLblmap


In [13]:
def crop_vol_and_mask_to_ROI (input_volume_map,referenceVolumeNode,segmentation_lable_map):

    #creating segmentation node to compute ROI 
    temp_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    temp_segmentationNode.CreateDefaultDisplayNodes()
    
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode (input_volume_map,temp_segmentationNode)
    
    
    
    ## obtaining bounding box coordinates from segment node
    import SegmentStatistics
    segStatLogic = SegmentStatistics.SegmentStatisticsLogic()
    segStatLogic.getParameterNode().SetParameter("Segmentation", temp_segmentationNode.GetID())
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_origin_ras.enabled",str(True))
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_diameter_mm.enabled",str(True))
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_x.enabled",str(True))
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_y.enabled",str(True))
    segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_z.enabled",str(True))
    segStatLogic.computeStatistics()
    stats = segStatLogic.getStatistics()


    ## creating ROI from bounding box stats

    segmentId = stats["SegmentIDs"] [0]
    print (segmentId)
    # Get bounding box
    obb_origin_ras = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_origin_ras"])
    obb_diameter_mm = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_diameter_mm"])
    obb_direction_ras_x = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_x"])
    obb_direction_ras_y = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_y"])
    obb_direction_ras_z = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_z"])
    
    # Create ROI
    segment = temp_segmentationNode.GetSegmentation().GetSegment(segmentId)
    roi=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
    roi.SetName(segment.GetName() + " OBB")
    roi.GetDisplayNode().SetHandlesInteractive(False)  # do not let the user resize the box
    roi.SetSize(obb_diameter_mm)
    
    
    # Position and orient ROI using a transform
    obb_center_ras = obb_origin_ras+0.5*(obb_diameter_mm[0] * obb_direction_ras_x + obb_diameter_mm[1] * obb_direction_ras_y + obb_diameter_mm[2] * obb_direction_ras_z)
    boundingBoxToRasTransform = np.row_stack((np.column_stack((obb_direction_ras_x, obb_direction_ras_y, obb_direction_ras_z, obb_center_ras)), (0, 0, 0, 1)))
    boundingBoxToRasTransformMatrix = slicer.util.vtkMatrixFromArray(boundingBoxToRasTransform)
    roi.SetAndObserveObjectToNodeMatrix(boundingBoxToRasTransformMatrix)


    ##easier and more reliable cropping method 
    cropped_volume_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
    cropped_volume_node.CreateDefaultDisplayNodes()
    
    cropped_lablemap_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    cropped_lablemap_node.CreateDefaultDisplayNodes()
    
    slicer.modules.cropvolume.logic().CropVoxelBased (roi,referenceVolumeNode,cropped_volume_node ) ##cropping volume 
    slicer.modules.cropvolume.logic().CropVoxelBased (roi,segmentation_lable_map,cropped_lablemap_node ) ##cropping mask


    return cropped_volume_node,cropped_lablemap_node


    

In [14]:
maxillary_teeth_lables = ["11","12","13","14","15","16","17","18","21","22","23","24","25","26","27","28"]
# maxillary_left_teeth_lables = ["11","12","13","14","15","16","17","18"]
# maxillary_right_teeth_lables = ["21","22","23","24","25","26","27","28"]
mandibular_teeth_lables = ["31","32","33","34","35","36","37","38","41","42","43","44","45","46","47","48"]

def convert_single_case_to_nii (dicom_series_path,segmentation_path):

    '''
    simply loading dicoms into a fresh scene 
    '''
    
    
    # # Clear scene
    slicer.mrmlScene.Clear(False)

    print(dicom_series_path)
    print(segmentation_path)
    
    ##load dicom 
    loadedNodeIDs = []
    with DICOMUtils.TemporaryDICOMDatabase() as db:
        DICOMUtils.importDicom(os.path.abspath(dicom_series_path), db)
        patientUIDs = db.patients()
    
        print (patientUIDs)
        for patientUID in patientUIDs:
            loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))



    '''
    loading the segmentations into appropriate substructuresz 
    
    '''

    maxillaryTeethNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    maxillaryTeethNode.CreateDefaultDisplayNodes()
    maxillaryTeethNode.SetName ("maxillaryTeethNode")

    
    ## creating segmentation nodes
    segmentationNodes = {} 


    ## importing stls 
    nodes = []
    node_names = []
    for seg in os.listdir(segmentation_path):
        if ((".stl" in seg or ".obj" in seg) and "pulp" not in seg ):
            mesh_path = f"{segmentation_path}/{seg}"
            class_name = get_model_class_from_file(seg)

            if class_name.strip("tooth_") in maxillary_teeth_lables:
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), maxillaryTeethNode)

            if class_name.strip("tooth_") in maxillary_teeth_lables:
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), maxillaryTeethNode)

            if class_name in lable_map:

                segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
                segmentationNode.CreateDefaultDisplayNodes()
                segmentationNodes [class_name] = segmentationNode
                
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), segmentationNode)

    if (len(segmentationNodes) <1):   
        print ("too few of desired class in this case. this case will be skipped")
        return;

    referenceVolumeNode = slicer.mrmlScene.GetNodeByID("vtkMRMLScalarVolumeNode1")

    segmentation_lable_map = convert_to_nnunet_lable_tooth(segmentationNodes,referenceVolumeNode)
    input_volume_map = convert_to_nnunet_lable_macro_structure(maxillaryTeethNode,referenceVolumeNode)

    cropped_input_volume_map, cropped_segmentation_lable_map= crop_vol_and_mask_to_ROI (input_volume_map,referenceVolumeNode,segmentation_lable_map)

    caseID = dicom_series_path.split("\\")[-1].split("_")[0]
    volume_save_path = f"{nii_path_volumes}/{caseID}_input.nii.gz"
    seg_save_path =  f"{nii_path_segmentations}/{caseID}_seg.nii.gz"

    slicer.util.exportNode(cropped_input_volume_map, volume_save_path , {"useCompression": 0})
    slicer.util.exportNode(cropped_segmentation_lable_map, seg_save_path , {"useCompression": 0})

    print ("one case sucessfully converted")
     


        

In [15]:
def batch_nii_convert (in_case_files):

    for case in tqdm(in_case_files[181:]):
        try:
            convert_single_case_to_nii (case["vol"],case["seg"])
        except:
            print (f"somthing is wrong with case {case} probably a missing segmentation")
    print ("done converting all cases to nii")
    

In [16]:
batch_nii_convert (input_cases)

  0%|          | 0/57 [00:00<?, ?it/s]

./temp_unzip\576376_volumes
./temp_unzip\576376_segmentations
('1',)
576376P524333_1536431790-cbct-tooth-18_decimated
one case sucessfully converted


  2%|1         | 1/57 [00:19<18:04, 19.36s/it]

./temp_unzip\576383_volumes
./temp_unzip\576383_segmentations
('1',)
576383P524337_576383_1441344234-cbct-tooth-17_decimated
one case sucessfully converted


  4%|3         | 2/57 [00:35<16:15, 17.75s/it]

...

./temp_unzip\579057_volumes
./temp_unzip\579057_segmentations
('1',)
579057P527586_1956671222-cbct-tooth-13_decimated
one case sucessfully converted


 98%|#########8| 56/57 [1:09:47<01:45, 105.58s/it]

./temp_unzip\579169_volumes
./temp_unzip\579169_segmentations
('1',)
somthing is wrong with case {'vol': './temp_unzip\\579169_volumes', 'seg': './temp_unzip\\579169_segmentations'} probably a missing segmentation


100%|##########| 57/57 [1:09:52<00:00, 73.56s/it] 


done converting all cases to nii


In [17]:
# test_seg = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\304846_246033_segmented_structures"
# test_vol = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\304846_Ken_Lin_Lin_Ken____20200831_080509147"

# test_seg2 = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\temp_unzip\\576547_segmentations"
# test_vol2 = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\temp_unzip\\576547_volumes"

# # # # convert_single_case_to_nii (input_cases[0]["vol"],input_cases[0]["seg"])
# convert_single_case_to_nii (test_vol2,test_seg2)

# building nnunet_dataset from nii

In [18]:
#making dataset Folder 

dataset_id = 130
dataset_name = "maxillary_left_teeth_split_from_volume" 
Dataset_folder_name = f"Dataset{dataset_id}_{dataset_name}"

if (not os.path.exists(f"./nnUNet_raw/{Dataset_folder_name}")):
    os.mkdir(f"./nnUNet_raw/{Dataset_folder_name}")

train_input_dir = f"./nnUNet_raw/{Dataset_folder_name}/imagesTr"
train_lable_dir = f"./nnUNet_raw/{Dataset_folder_name}/labelsTr"

if not os.path.exists(train_input_dir):
    os.mkdir(train_input_dir)
if not os.path.exists(train_lable_dir):
    os.mkdir(train_lable_dir)




In [19]:
import shutil
volume_list = os.listdir(nii_path_volumes)
seg_list = os.listdir(nii_path_segmentations)
# print (volume_list)

for volume in volume_list:
    volume_old_path = f"{nii_path_volumes}/{volume}"
    volume_nnunet_name = volume.replace ("_input.nii.gz","_0000.nii.gz")
    volume_new_path = f"{train_input_dir}/{volume_nnunet_name}"

    #print (volume_nnunet_name)
    shutil.move(volume_old_path, volume_new_path)


for segmentation in seg_list:
    segmentation_old_path = f"{nii_path_segmentations}/{segmentation}"
    segmentation_nnunet_name = segmentation.replace ("_seg.nii.gz",".nii.gz")
    segmentation_new_path = f"{train_lable_dir}/{segmentation_nnunet_name}"

    #print (segmentation_nnunet_name)
    shutil.move(segmentation_old_path, segmentation_new_path)


In [20]:
## creating Dataset Json 
import json 

dataset_metadata = {
 "channel_names": {  # formerly modalities
   "0": "CT", 
 }, 
 "labels": {
    "background":0,
    "tooth_11":1,
    "tooth_12":2,
    "tooth_13":3,
    "tooth_14":4,
    "tooth_15":5,
    "tooth_16":6,
    "tooth_17":7,
    "tooth_18":8,
    # "tooth_21":9,
    # "tooth_22":10,
    # "tooth_23":11,
    # "tooth_24":12,
    # "tooth_25":13,
    # "tooth_26":14,
    # "tooth_27":15,
    # "tooth_28":16,

}, 
 "numTraining": len(volume_list), 
 "file_ending": ".nii.gz",
 "overwrite_image_reader_writer": "SimpleITKIO"  # optional! If not provided nnU-Net will automatically determine the ReaderWriter
 
}

json_data = json.dumps(dataset_metadata)

with open(f"./nnUNet_raw/{Dataset_folder_name}/dataset.json", "w") as outfile:
    outfile.write(json_data)

In [21]:
# for file in os.listdir(train_input_dir):

#     volume_nnunet_name = file.replace ("_input.nii.gz","_0000.nii.gz")
#     os.rename (f"{train_input_dir}/{file}",f"{train_input_dir}/{volume_nnunet_name}")

# tests


In [22]:
dicom_series_path = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\304846_Ken_Lin_Lin_Ken____20200831_080509147"
segmentation_path = "304846_246033_segmented_structures"

In [23]:
# # Clear scene
slicer.mrmlScene.Clear(False)

print(dicom_series_path)
print(segmentation_path)

##load dicom 
loadedNodeIDs = []
with DICOMUtils.TemporaryDICOMDatabase() as db:
    DICOMUtils.importDicom(os.path.abspath(dicom_series_path), db)
    patientUIDs = db.patients()

    print (patientUIDs)
    for patientUID in patientUIDs:
        loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))





C:\Users\Sepehr\3dsegmentationProjects\nn-unet-organize-dataset-for-training\304846_Ken_Lin_Lin_Ken____20200831_080509147
304846_246033_segmented_structures
('1',)


In [24]:
'''
loading the segmentations into appropriate substructuresz 

'''
maxillary_teeth_lables = ["11","12","13","14","15","16","17","18","21","22","23","24","25","26","27","28"]
maxillaryTeethNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
maxillaryTeethNode.CreateDefaultDisplayNodes()
maxillaryTeethNode.SetName ("maxillaryTeethNode")


## creating segmentation nodes
segmentationNodes = {} 


In [20]:
## importing stls 
nodes = []
node_names = []
for seg in os.listdir(segmentation_path):
    if ((".stl" in seg or ".obj" in seg) and "pulp" not in seg ):
        mesh_path = f"{segmentation_path}/{seg}"
        class_name = get_model_class_from_file(seg)

        if class_name.strip("tooth_") in maxillary_teeth_lables:
            slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), maxillaryTeethNode)

        if class_name in lable_map:

            segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
            segmentationNode.CreateDefaultDisplayNodes()
            segmentationNodes [class_name] = segmentationNode
            
            slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), segmentationNode)

if (len(segmentationNodes) <1):
    print ("too few of desired class in this case. this case will be skipped")
    

In [ ]:
maxillaryTeethNode.

In [34]:
referenceVolumeNode = slicer.mrmlScene.GetNodeByID("vtkMRMLScalarVolumeNode1")

segmentation_lable_map = convert_to_nnunet_lable_tooth(segmentationNodes,referenceVolumeNode)
input_volume_map = convert_to_nnunet_lable_macro_structure(maxillaryTeethNode,referenceVolumeNode)

In [55]:
#creating segmentation node to compute ROI 
temp_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
temp_segmentationNode.CreateDefaultDisplayNodes()

slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode (input_volume_map,temp_segmentationNode)



## obtaining bounding box coordinates from segment node
import SegmentStatistics
segStatLogic = SegmentStatistics.SegmentStatisticsLogic()
segStatLogic.getParameterNode().SetParameter("Segmentation", temp_segmentationNode.GetID())
segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_origin_ras.enabled",str(True))
segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_diameter_mm.enabled",str(True))
segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_x.enabled",str(True))
segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_y.enabled",str(True))
segStatLogic.getParameterNode().SetParameter("LabelmapSegmentStatisticsPlugin.obb_direction_ras_z.enabled",str(True))
segStatLogic.computeStatistics()
stats = segStatLogic.getStatistics()



In [56]:
## creating ROI from bounding box stats

segmentId = stats["SegmentIDs"] [0]
print (segmentId)
# Get bounding box
obb_origin_ras = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_origin_ras"])
obb_diameter_mm = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_diameter_mm"])
obb_direction_ras_x = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_x"])
obb_direction_ras_y = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_y"])
obb_direction_ras_z = np.array(stats[segmentId,"LabelmapSegmentStatisticsPlugin.obb_direction_ras_z"])

# Create ROI
segment = temp_segmentationNode.GetSegmentation().GetSegment(segmentId)
roi=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
roi.SetName(segment.GetName() + " OBB")
roi.GetDisplayNode().SetHandlesInteractive(False)  # do not let the user resize the box
roi.SetSize(obb_diameter_mm)


# Position and orient ROI using a transform
obb_center_ras = obb_origin_ras+0.5*(obb_diameter_mm[0] * obb_direction_ras_x + obb_diameter_mm[1] * obb_direction_ras_y + obb_diameter_mm[2] * obb_direction_ras_z)
boundingBoxToRasTransform = np.row_stack((np.column_stack((obb_direction_ras_x, obb_direction_ras_y, obb_direction_ras_z, obb_center_ras)), (0, 0, 0, 1)))
boundingBoxToRasTransformMatrix = slicer.util.vtkMatrixFromArray(boundingBoxToRasTransform)
roi.SetAndObserveObjectToNodeMatrix(boundingBoxToRasTransformMatrix)

304846_246033_6971_304846_tooth_11_(us_8)_decimated


In [57]:
## function to retrieve segments of volume using ROI 
# import vtk
# def createVolumeFromRoi(exportRoi, spacingMm, fillValue=0, numberOfComponents=1):
#     import math
#     roiDiameter = exportRoi.GetSize()
#     roiOrigin_Roi = [-roiDiameter[0]/2, -roiDiameter[1]/2, -roiDiameter[2]/2, 1]
#     roiToRas = exportRoi.GetObjectToWorldMatrix()
#     exportVolumeSize = [int(math.ceil(diameterComponent/spacingMm)) for diameterComponent in roiDiameter]
#     # Create image data
#     exportImageData = vtk.vtkImageData()
#     exportImageData.SetExtent(0, exportVolumeSize[0]-1, 0, exportVolumeSize[1]-1, 0, exportVolumeSize[2]-1)
#     exportImageData.AllocateScalars(vtk.VTK_DOUBLE, numberOfComponents)
#     exportImageData.GetPointData().GetScalars().Fill(fillValue)
#     # Create volume node
#     exportVolume = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode" if numberOfComponents==1 else "vtkMRMLVectorVolumeNode")
#     exportVolume.SetAndObserveImageData(exportImageData)
#     exportVolume.SetIJKToRASDirections(roiToRas.GetElement(0,0), roiToRas.GetElement(0,1), roiToRas.GetElement(0,2), roiToRas.GetElement(1,0), roiToRas.GetElement(1,1), roiToRas.GetElement(1,2), roiToRas.GetElement(2,0), roiToRas.GetElement(2,1), roiToRas.GetElement(2,2))
#     exportVolume.SetSpacing(spacingMm, spacingMm, spacingMm)
#     roiOrigin_Ras = roiToRas.MultiplyPoint(roiOrigin_Roi)
#     exportVolume.SetOrigin(roiOrigin_Ras[0:3])
#     return exportVolume

In [58]:
# cropVolumeParameters = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLCropVolumeParametersNode")
# cropVolumeParameters.SetInputVolumeNodeID(referenceVolumeNode.GetID())
# cropVolumeParameters.SetROINodeID(roi.GetID())
# slicer.modules.cropvolume.logic().SnapROIToVoxelGrid(cropVolumeParameters)  # optional (rotates the ROI to match the volume axis directions)
# slicer.modules.cropvolume.logic().FitROIToInputVolume(cropVolumeParameters)
# slicer.mrmlScene.RemoveNode(cropVolumeParameters)

In [61]:
##easier and more reliable cropping method 
cropped_volume_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
cropped_volume_node.CreateDefaultDisplayNodes()

cropped_lablemap_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
cropped_lablemap_node.CreateDefaultDisplayNodes()

slicer.modules.cropvolume.logic().CropVoxelBased (roi,referenceVolumeNode,cropped_volume_node ) ##cropping volume 
slicer.modules.cropvolume.logic().CropVoxelBased (roi,segmentation_lable_map,cropped_lablemap_node ) ##cropping mask

0

In [63]:
dicomData = slicer.util.arrayFromVolume(cropped_volume_node)
mask = slicer.util.arrayFromVolume(cropped_lablemap_node)

print (mask.shape)
print (dicomData.shape)

(260, 459, 586)
(260, 459, 586)


In [64]:
caseID = dicom_series_path.split("\\")[-1].split("_")[0]
volume_save_path = f"{nii_path_volumes}/{caseID}_input.nii.gz"
seg_save_path =  f"{nii_path_segmentations}/{caseID}_seg.nii.gz"

slicer.util.exportNode(cropped_volume_node, volume_save_path , {"useCompression": 0})
slicer.util.exportNode(cropped_lablemap_node, seg_save_path , {"useCompression": 0})

print ("one case sucessfully converted")

one case sucessfully converted
